In [ ]:
from utils import *
import tensorflow as tf
from keras import backend as K

In [ ]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
K._get_available_gpus()

In [ ]:
tickers_sets_path = 'data/tickers/sets/'

sets_files = sorted([tickers_sets_path + x for x in listdir(tickers_sets_path) if 'json' in x], reverse=True)

tickers_sets = []

for set_file in sets_files:
    with open(set_file, 'r') as f:
        tickers_sets.append(json.load(f))

dst_path = 'data/stocks/'

In [ ]:
df = pd.read_csv('data/dataset.csv')
df = df.set_index('Date')
df = df[:-80]

In [ ]:
data_size = 4800
input_tickers = tickers_sets[0]
output_tickers = tickers_sets[0]

In [ ]:
scaler, x, y, dic = dataa(df, data_size, input_tickers, output_tickers, step_size=1, input_size=60, output_size=20, feature_range=(0,1))

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = train_val_test_split(x,y)

In [ ]:
input_shape = (x.shape[1],x.shape[2])
output_shape = (y.shape[1], y.shape[2])
layers_info_list = [
    [
        (50, 0.2, True), 
        (70, 0.2, True),
        (70, 0.2, True),
        (60, 0.2, True),
        (40, 0.2, False)
    ],
    [
        (100, 0.2, True), 
        (100, 0.2, True),
        (100, 0.2, True),
        (150, 0.2, False)
    ],
    [
        (450, 0.2, True), 
        (600, 0.2, True),
        (600, 0.2, True),
        (400, 0.2, False)
    ],
    [
        (50, 0.2, True), 
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, True),
        (50, 0.2, False)
    ],
    [
        (85, 0.2, True), 
        (85, 0.2, True),
        (85, 0.2, True),
        (85, 0.2, True),
        (85, 0.2, True),
        (85, 0.2, True),
        (85, 0.2, True),
        (85, 0.2, True),
        (85, 0.2, False)
    ]
]


In [ ]:
layers_info_list = [
    [
        (600, 0.2, True), 
        (800, 0.2, True),
        (800, 0.2, True),
        (600, 0.2, False)
    ],
    [
        (1000, 0.15, True), 
        (1000, 0.15, True),
        (1000, 0.15, False)
    ],
    [
        (800, 0.1, True), 
        (1000, 0.1, True),
        (1000, 0.1, True),
        (800, 0.1, False)
    ]
]

In [ ]:
models = []

for layers_info in layers_info_list:

    models.append(create_model(input_shape, output_shape, layers_info, metrics=['accuracy', 'mse']))

In [ ]:
history_list = []
epochs = 100

i = 5
set_name = 'set6/'
plot_path_dir = 'plots/' + set_name

for model in models:
    
    try:
        
        history = model.fit(
            x=x_train,
            y=y_train,
            epochs=epochs,
            batch_size=32,
            validation_data=(x_val, y_val)
        )

        history_list.append(history)

        model_name = 'LSTM' + str(i)
        model.save('models/' + set_name + model_name + '/model')

        plot_path_loss = plot_path_dir + 'LOSS_' + model_name + '.png'
        validation_plot(history.history['loss'], history.history['val_loss'], 'train vs validation loss', f=lambda: plt.savefig(plot_path_loss))

        plot_path_mse = plot_path_dir + 'MSE_' + model_name + '.png'
        validation_plot(history.history['mse'], history.history['val_mse'], 'train vs validation mse', f=lambda: plt.savefig(plot_path_mse), plot_type='mse')

        plot_path_accuracy = plot_path_dir + 'ACCURACY_' + model_name + '.png'
        validation_plot(history.history['accuracy'], history.history['val_accuracy'], 'train vs validation accuracy', f=lambda: plt.savefig(plot_path_accuracy), plot_type='accuracy')

    except:

        print('ERRO NO MODELO ' + str(i))
    
    
    i += 1